# Homework 3

For this homework we have used cloud technologies (Google Colab) in order to have enough memory to handle the dataset and to store the tsv files without saving them on our personal computers

In [ ]:
from google.colab import drive, files
import sys
import nltk
import pickle

drive.mount('/content/drive/')
sys.path.append('/content/drive/My Drive/ADM-HW3/HW3')

nltk.download('stopwords')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1. Data collection

In [ ]:
from Parse_Urls import write_url
from FilesLoad import loadPages
from Html_Parse import analyse_dir
import Search_engine as se
from Search_engine import Search_engine2 as se2
from Search_engine import Search_engine3 as se3

### 1.1 and 1.2
We download the html pages using paralleling process and we decided to devide the subset of pages to reduce the execution time for this step. To do this, each member of the group pass to *loadHtmls* the start and end number to delimit its subset of pages.<br>
Each page is downloaded after 5 seconds from the previous to avoid the downloads blocking by the site.

In [ ]:
# It creates the file contains all the 20000 links
write_url("/content/drive/My Drive/ADM-HW3/HW3")

# It downloads all the links from the start to the end (the end is not included)
def loadHtmls(start=1,end=20001):
    loadPages(start, end)

loadHtmls()
print("All the pages have been downloaded")

All the pages have been downloaded


### 1.3
In this section we parse all the downloaded html pages in order to obtain the requested fields, to do that we iterate on the folders pages (each page contains 50 anime) and we parse the html pages in these folders.

In [ ]:
for i in range(1,384):
    analyse_dir(i)
print("All the pages have been parsed")

All the pages have been parsed


## 2. Search Engine

In order to improve the speed of our work we decided to save the preprocessing of the anime documents instead of redoing it each time. In particular we: tokenize strings, remove the stopwords and the punctuation and finally we stem the words. All of this is not done on all the fields (e.g dates and numbers are not preprocessed)

In [ ]:
p = se.preprocessing_files()
print("All the files have been preprocessed")

All the files have been preprocessed


### 2.1
It computes the conjunctive query creating two dictionaries: vocabulary and inverted index. We save them using the pickle module in order to access them faster. It prints the result of the query.<br>
The parameters for the function are:
- query: text we want to search;
- check: boolean to indicate whether we want to print the results or not;
- field: boolean to indicate whether we want:
    - True: searching only into animeDescription section
    - False: searching on all the document sections.

In [ ]:
query = "Return hope"
k = 5

In [ ]:
se.And_query(query, True, True)

title, Shingeki no Kyojin Season 3 Part 2
description,  Seeking to restore humanity's diminishing hope, the Survey Corps embark on a mission to retake Wall Maria, where the battle against the merciless "Titans" takes the stage once again.                   Returning to the tattered Shiganshina District that was once his home, Eren Yeager and the Corps find the town oddly unoccupied by Titans. Even after the outer gate is plugged, they strangely encounter no opposition. The mission progresses smoothly until Armin Arlert, highly suspicious of the enemy's absence, discovers distressing signs of a potential scheme against them.            <i>        Shingeki no Kyojin Season 3 Part 2        </i>        follows Eren as he vows to take back everything that was once his. Alongside him, the Survey Corps strive—through countless sacrifices—to carve a path towards victory and uncover the secrets locked away in the Yeager family's basement.                   [Written by MAL Rewrite]
url,  https:/

### 2.2
It computes the conjunctive query recalling the previous And_query funcion in order to obtain a list of the documents in wich the words in the query appear. The function also create another reverted index containing, for document id, a list of tuples (id_term, tdfIdf). After that our function invoke another function to compute cosine similarity starting from: vocabulary, inverted index, query and the new index that we just created, then it prints the results.<br>
The parameters for the function are:
- query: text we want to search;
- field: boolean to indicate whether we want as above;
- check: boolean to indicate whether we want to print the results or not;
-k: the number of results to print.

In [ ]:
se2.Query_AndFreq(query, True, True, k)

title, Free! (Free! - Iwatobi Swim Club)
description,  Haruka Nanase has a love for water and a passion for swimming. In elementary school, he competed in and won a relay race with his three friends Rin Matsuoka, Nagisa Hazuki, and Makoto Tachibana. After claiming victory at the tournament, the four friends went their separate ways. Years later, they reunite as high school students; however, Rin couldn't care less about returning to the way things used to be. Not only does he attend a different school, but the sole thing important to him is proving that he is a better swimmer than Haruka.                   After the bitter reunion, Haruka, Nagisa, and Makoto decide to form the Iwatobi High School Swim Club, but they will need a fourth member if they hope to take part in the upcoming tournament. Enter Rei Ryuugazaki, a former member of the track team whom Nagisa recruits. As the time to compete draws near, the four develop a close bond while training intensely to come out on top and set

## 3. Define a new score

We defined a new score function that takes in input the result of the previous conjunctive query (with cosine similarity), the string we want to search and the number of results we want to visualize.<br>
Our new scoring function takes into account where the words in the query are so we assing a different score based on the section in wich it appears in the anime as follows:
- 10 points to a word found in the anime title;
- 8 points to a word found animeCharacters section;
- 5 point to a word found in the staff section;
- 4 point to a word found in the voices section.

Than, in case of a tie on the new score we also sort them based on their rank on myanimelist.net thus offering to the user results from the best to the worst.

In [ ]:
query2 = "gintama sakata"

# We recall this function because, thanks to different flags, we are able to define new inverted index
# and vocabulary with all the words that appear in all the fields of html pages and not just those that
# appear in animeDescription section
_ = se.And_query(query2, False, False)

In [ ]:
k = 10

cos = se2.Query_AndFreq(query2, False,False, k)
se3.rank(cos, query2, k)

title, Gintama° (Gintama Season 4)
description,  Gintoki, Shinpachi, and Kagura return as the fun-loving but broke members of the Yorozuya team! Living in an alternate-reality Edo, where swords are prohibited and alien overlords have conquered Japan, they try to thrive on doing whatever work they can get their hands on. However, Shinpachi and Kagura still haven't been paid... Does Gin-chan really spend all that cash playing pachinko?                   Meanwhile, when Gintoki drunkenly staggers home one night, an alien spaceship crashes nearby. A fatally injured crew member emerges from the ship and gives Gintoki a strange, clock-shaped device, warning him that it is incredibly powerful and must be safeguarded. Mistaking it for his alarm clock, Gintoki proceeds to smash the device the next morning and suddenly discovers that the world outside his apartment has come to a standstill. With Kagura and Shinpachi at his side, he sets off to get the device fixed; though, as usual, nothing is e

[(2, 28),
 (6, 28),
 (9, 28),
 (95, 28),
 (10, 18),
 (15, 18),
 (16, 18),
 (22, 18),
 (1801, 18),
 (5002, 18)]

In the output of the above cell you can see, at the end, a list containing the document id and the score associated with that document. With this list is clear that, after ordering with respect to score, there is another sorting with respect to the rank of each anime.

## 5. Algorithmic question
We used Dynamic Programming in order to solve the problem.
Dynamic Programming is a computer programming method to optimize recursion.
It breaks down a complicated problem into simpler sub-problems so that
we don’t have to re-compute them when needed later.


### 5.1

Take a list as input we take its length and if there are no elements we return 0, if there is only one element we return this element.<br>
In the more complex case we use a for loop to consider whether it is convenient to take the element in position i or not and we store this choice in the dynamic programming table (actually we store the value obtained by making the choice). Then we retrieve the optimal "path" using a while loop that "returns" the reversed path, so we reverse the list to obtain the correct path to return.<br>
Below you can see the pseudocode.

In [ ]:
# PSEUDOCODE
def solve(nums):
    N = length of nums list
    if N == 0: return N
    if N == 1: return nums[0]

    dp = list of N+1 elements initialized to 0
    dp[0] = nums[0]
    for i = 2 to N+1:
        dp[i] = max(dp[i-2] + nums[i-1], dp[i-1])
    
    result = an empty list that will contain the final result
    j = N
    while j > 0:
        if j > 1:
            if dp[j-1] > (dp[j-2] + nums[j-1]):
                j--
            else:
                result.append(nums[j-1])
                j -= 2
        else if j==1 && dp[j-1] < nums[j-1]:
            result.append(nums[j-1])
            j--
        else:
            j--
    return result reversed

### 5.2

In [ ]:
def solve(nums):
    N = len(nums)
    # If there is no request, you can’t make an appoinment
    if not nums:
        return 0
    # Ff there is only one request, you can only fix that request
    if N==1:
        return nums[0]
    # If there are two or more requests, you will try to set the maximum amount of time without making appoinment
    # from the two adjacent request.
    # We define an empty list with the size of our input and we insert the first element of the input
    # in the second position of the list for the sake of comparison.
    dp = [None for _ in range(N + 1)]
    dp[0] = 0
    dp[1] = nums[0]

    # We define a loop for comparing two different paths. This path starts with an element that we decide to whether
    # pick it or not. The last item in the list is the biggest summation we could get out of the input.
    for i in range(2, N + 1):
        dp[i] = max(dp[i-2] + nums[i-1], dp[i-1])

    # Now we define a loop for extracting indices of the elements that produced the biggest summation
    result = []
    j = N
    while j > 0:
        if j > 1:
            if dp[j - 1] > dp[j - 2] + nums[j - 1]:
                j = j - 1
            else:
                result.append(nums[j - 1])
                j = j - 2
        elif j==1 and dp[j-1] < nums[j-1]:
            result.append(nums[j-1])
            j = j-1
        else:
            j = j-1
    result.reverse()
    s = sum(result)
    return result, sum(result)

# Testing the algorithm:
nums = [30, 40, 25, 50, 30, 20]
vec, sum = solve(nums)
print(f"The optimal solution is: {vec}")
print(f"The biggest duration possible is: {sum}")



The optimal solution is: [40, 50, 20]
The biggest duration possible is: 110
